In [2]:

import pandas as pd
import json
from pathlib import Path
import numpy as np
import json
 
    

def load_data(data_file="MRC_with_disc/train.json"):
    #TODO: save model checkpoint together with data    
    data_file = Path(data_file)
    with open (data_file) as f: 
        data = json.load(f)
    return data


def process_molweni(data):
    mydata= []
    for d in data["data"]["dialogues"]:
        dial = {}

        #all utterances in the dialogue
        context = []
        for line in d['edus']:
            context.append(line['text'])
        dial["context"] = context

        #last utterance
        dial["answer"] = d['edus'][-1]['text']

        #utterance to which the last utterance replies
        dial["ans_idx"] = d['relations'][-1]['x']

        #list of relations [[speaker id, addressee id]]
        relation_at = []
        for line in d['relations']:
            relation_at.append([line['y'], line['x']])
        dial["relation_at"] = relation_at

        #pull users list
        users = []
        for line in d['edus']:
            users.append(line['speaker'])

        usersnp = np.array(users)

        #build user relations
        user_rels  = []
        for u in set(usersnp):
            user_rel = np.where(usersnp == u)[0].tolist()
            if len(user_rel)>1: 
                user_rel_list = list(user_rel)
                user_rel_list.sort(reverse=True)
                user_rels.append(user_rel_list)
        dial["relation_user"] = user_rels 
    
        mydata.append(dial)
    return mydata

def process_data_and_write(dtype="train"):
    Path("./mydata").mkdir(exist_ok=True) 
    data_path = f"MRC_with_disc/{dtype}.json"
    data = load_data(data_file=data_path)
    mydata = process_molweni(data)
    json_str = json.dumps(mydata, indent=4) + '\n'
    Path(f"mydata/{dtype}.json").write_text(json_str, encoding='utf-8')

def process_data_and_write(dtype="train"):
    Path("./mydata").mkdir(exist_ok=True) 
    data_path = f"MRC_with_disc/{dtype}.json"
    data = load_data(data_file=data_path)
    mydata = process_molweni(data)
    json_str = json.dumps(mydata, indent=4) + '\n'
    Path(f"mydata/{dtype}.json").write_text(json_str, encoding='utf-8')
    
for d in ["test","train","dev"]: 
    process_data_and_write(dtype=d)